## OPRACOWANIE DANYCH ETAP 2

W pierwszym etapie projektu zespołowego została przygotowana wstępna wersja danych, w której dokonano wyboru okresu czasowego oraz języka badanych piosenek.

W tym etapie zostanie przygotowana analiza danych, która pozwoli podzielić piosenki na kategorie. Informacje o kategoriach będą wykorzystane w kolejnym etapie do stworzenia modelu predykcyjnego.

In [ ]:
%cd ../
%pip install -r requirements.txt
%cd data_preparing

In [151]:
#import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from text2emotion import get_emotion
from nltk.sentiment.vader import SentimentIntensityAnalyzer

Po wczytaniu potrzebnych bibliotek, wczytujemy dane z pliku csv.

In [152]:
df = pd.read_csv('data.csv')

df.shape
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 385269 entries, 0 to 385268
Data columns (total 9 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   title     385266 non-null  object
 1   tag       385269 non-null  object
 2   artist    385269 non-null  object
 3   year      385269 non-null  int64 
 4   views     385269 non-null  int64 
 5   features  385269 non-null  object
 6   lyrics    385269 non-null  object
 7   id        385269 non-null  int64 
 8   decade    385269 non-null  object
dtypes: int64(3), object(6)
memory usage: 26.5+ MB


In [154]:
df.head()

,title,tag,artist,year,views,features,lyrics,id,decade
0,Can I Live,rap,JAY-Z,1996,468624,{},"[Produced by Irv Gotti]\n\n[Intro]\nYeah, hah, yeah, Roc-A-Fella\nWe invite you to somethin' epic, you know?\nWell, we hustle out of a sense of hopelessness\nSort of a desperation\nThrough that desperation, we become addicted\nSort of like the fiends we accustomed to servin'\nBut we feel we have nothin' to lose\nSo, we offer you, well, we offer our lives, right?\nWhat do you bring to the table?\n\n[Verse 1]\nWhile I'm watchin' every nigga watchin' me closely\nMy shit is butter for the bread, they wanna toast me\nI keep my head, both of them, where they supposed to be\nHoes'll get you sidetracked, then clapped from close feet\nI don't sleep, I'm tired, I feel wired like codeine, these days\nA brother gotta admire me from four fiends away\nMy pain, wish it was quick to see\nFrom sellin' 'caine 'til brains was fried to a fricassee\nCan't lie, at the time it never bothered me\nAt the bar, gettin' my thug on properly\nMy squad and me lack of respect for authority\nLaughin' hard, happy t...",3,1990s
1,Rockin and Rollin,rap,Cam'ron,1998,6399,"{""Cam\\'ron""}","[Verse 1]\nAy yo you wonder who I are\nI guzzle up at the bar\nBut you see me in the cars that start with the double R\nRange Rover, Rolls Royce, even got a Rocky Rolly\nSee more ice than a hockey goalie\nBaby eating guacamole\nI did without\nNow I live it out\nCars? got to whip it out\nEvery year I get it out\n(Why?) I be long gone\n(Where?) probably Hong Kong\nGirls with their thong on\nI'm playing Don Juan\nDrinking Chandon\nWhere the chron, ma?\n""You smoke weed?"" ""What you think, girl? Uh-huh!""\nStart the car up\nTake her on a long ride\nYo she think my steering wheel on the wrong side\nNo I'm sorry\nThis is the Ferrari\nLimon like Bacardi\nRock with safari\nBut la di da di\nWe like to party\nAnd every night, believe, we gon' leave with somebody\n[Hook]\nWe was rockin' and rollin'\nNow we rollin' and smokin'\nOn the phone and we chokin'\nWhile you strollin' and hopin'\nFor the tone which you spoke in\nAnd I know that you're open\nLive a Branson life and a brand new 5\nHot man, ...",10,1990s
2,DEvils,rap,JAY-Z,1996,504959,{},"[Produced by DJ Premier]\n\n[Hook Samples: Snoop Dogg + Prodigy]\n""Dear God – I wonder, can you save me?""\n""Illuminati want my mind, soul and my body""\n""Dear God – I wonder, can you save me?""\n""Secret society, tryna keep they eye on me""\n""Dear God – I wonder, can you save me?""\n""Illuminati want my mind, soul and my body""\n""Dear God – I wonder, can you save me?""\n""I can't die, I can't die, I can't die""\n\n[Verse 1]\nThis shit is wicked on these mean streets\nNone of my friends speak, we're all tryna win\nBut then again, maybe it's for the best though\n'Cause when they're sayin' too much\nYou know they're tryna get you touched\nWhoever said illegal was the easy way out\nCouldn't understand the mechanics\nAnd the workings of the underworld, granted\nNine-to-five is how you survive, I ain't tryna survive\nI'm tryna live it to the limit and love it a lot\nLife ills poisoned my body, I used to say fuck mic skills\nI never prayed to God, I prayed to Gotti\nThat's right, it's wicked – that...",14,1990s
3,Its Hot Some Like It Hot,rap,JAY-Z,1999,103549,{},"[Produced by Timbaland]\n\n[Verse 1]\nYo, show closer, J-to-the-A-Y-Hovah\nPlace shutter down, who the fuck'll fuck around?\nGame spitter, Range sitter, Bentley driver nigga\nKeep a full clip, I have to empty out on niggas\nHo bagger, no slacker, get this shit jumping\nLike eight blacks, four crackers, get your ass jumped\nCris' sipper, six dipper, wrist glitter nigga\nGat buster, ass toucher.. clit licker\nGo against Jigga your ass is dense\nI'm about a dollar, what the fuck is 50 Cents?\nHot shit, kick a nigga, turn these mics out\nMy jewelry so bright you can turn these lights out\nHova's like Noah keep two in the truck\nI'm like U-Haul; every bitch moving I fuck\nYou move slut, I gotta put two i

Wstępna baza danych powinna być zrównoważona oraz nie za duża. Zostanie wybranych 3600 rekordów - po 200 z każdej dekady i gatunku.

In [153]:
df.groupby(['tag','decade']).size()

tag      decade
country  1970s       6878
         1980s       4553
         1990s       7264
misc     1970s       1067
         1980s       1018
         1990s       2765
pop      1970s      33574
         1980s      57186
         1990s     134911
rap      1970s        217
         1980s       3347
         1990s      21123
rb       1970s       6288
         1980s       5076
         1990s       6808
rock     1970s      17153
         1980s      25984
         1990s      50057
dtype: int64

In [155]:
def take_first_200(group):
    return group.head(200)

df_3600 = df.groupby(['decade', 'tag']).apply(take_first_200)
df_3600.reset_index(drop=True, inplace=True)

df = df_3600
df.head()

,title,tag,artist,year,views,features,lyrics,id,decade
0,Rocky Mountain High,country,John Denver,1972,34529,{},"[Verse 1]\nHe was born in the summer of his 27th year\nComin' home to a place he’d never been before\nHe left yesterday behind him, you might say he was born again\nYou might say he found a key for every door\n\n[Verse 2]\nWhen he first came to the mountains his life was far away\nOn the road and hangin’ by a song\nBut the string’s already broken and he doesn’t really care\nIt keeps changin' fast and it don’t last for long\n\n[Chorus]\nBut the Colorado Rocky Mountain high\nI’ve seen it rainin' fire in the sky\nThe shadow from the starlight is softer than a lullaby\nRocky Mountain high (Colorado)\nRocky Mountain high (High in Colorado)\n\n[Verse 3]\nHe climbed cathedral mountains, he saw silver clouds below\nHe saw everything as far as you can see\nAnd they say that he got crazy once, and he tried to touch the sun\nAnd he lost a friend but kept the memory\n[Verse 4]\nNow he walks in quiet solitude the forests and the streams\nSeeking grace in every step he takes\nHis sight has turne...",2873,1970s
1,New Kid in Town,country,Eagles,1976,76529,{},"[Verse 1]\nThere's talk on the street, it sounds so familiar\nGreat expectations, everybody's watching you\nPeople you meet, they all seem to know you\nEven your old friends treat you like you're something new\n\n[Chorus]\nJohnny-come-lately, the new kid in town\nEverybody loves you, so don't let them down\n\n[Verse 2]\nYou look in her eyes, the music begins to play\nHopeless romantics, here we go again\nBut after a while, you're lookin' the other way\nIt's those restless hearts that never mend (Oh)\n\n[Chorus]\nJohnny-come-lately, the new kid in town\nWill she still love you when you're not around?\n\n[Bridge]\nThere's so many things you should have told her\nBut night after night, you're willing to hold her, just hold her\nTears on your shoulder\n[Verse 3]\nThere's talk on the street, it's there to remind you\nDoesn't really matter which side you're on\nYou're walking away and they're talking behind you\nThey will never forget you 'til somebody new comes along\n\n[Chorus]\nWhere ...",54282,1970s
2,Pretty Maids All in a Row,country,Eagles,1976,21183,{},"[Verse 1]\nHi there, how are you? It's been a long time\nSeems like we've come a long way\nMy, but we learn so slow\nAnd heroes, they come and they go\nAnd leave us behind as if we're supposed to know\nWhy\nWhy\nWhy do we give up our hearts to the past?\nAnd why must we grow up so fast?\n\n[Verse 2]\nAnd all you wishing well fools with your fortunes\nSomeone should send you a rose\nWith love from a friend, it's nice to hear from you again\nAnd the storybook comes to a close\nGone are the ribbons and bows\nThings to remember, places to go\nPretty maids all in a row",54286,1970s
3,Try and Love Again,country,Eagles,1976,13921,{},"When you're out there on your own\nWhere your memories can find you\nLike a circle goes around\nYou were lost until you found out\nWhat it all comes down to\n\nOne by one the lonely feelings come\nDay by day they slowly fade away\n\nOoh, the look was in her eyes\nYou never know what might be found there\nShe was dancing right in time\nAnd the moves she made so fine\nLike the music that surrounds her\n\nShould I stay or go? I really want to know\nWould I lose or win if I try and love again?\n\nOh-oh, oh-oh, oh-oh-oh, gonna try and love again\nOh-oh, oh-oh, oh-oh-oh, I'm gonna try and love again\nOh-oh, oh-oh, oh-oh-oh, gonna try and love\nRight or wrong, what's done is done\nIt's only moments that you borrow\nBut the thoughts will linger on\nOf the lady and her song\nWhen the sun comes up tomorrow\nWell, it might take years\nTo see through all these tears\nDon't let go\nWhen you find it, you will know\n\nOh-oh, oh-oh, oh-oh-oh, gonna try and love again\nOh-oh, oh-oh, oh-oh-oh, gonna...",54287,1970s
4,Margaritaville,country,Jimmy Buffett,1977,144670,{},"[Verse 1]\nNibblin' on sponge cake\nWatchin' the sun b

In [156]:
df_3600.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3600 entries, 0 to 3599
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   title     3600 non-null   object
 1   tag       3600 non-null   object
 2   artist    3600 non-null   object
 3   year      3600 non-null   int64 
 4   views     3600 non-null   int64 
 5   features  3600 non-null   object
 6   lyrics    3600 non-null   object
 7   id        3600 non-null   int64 
 8   decade    3600 non-null   object
dtypes: int64(3), object(6)
memory usage: 253.3+ KB


In [157]:
df.groupby(['decade','tag']).size()

decade  tag    
1970s   country    200
        misc       200
        pop        200
        rap        200
        rb         200
        rock       200
1980s   country    200
        misc       200
        pop        200
        rap        200
        rb         200
        rock       200
1990s   country    200
        misc       200
        pop        200
        rap        200
        rb         200
        rock       200
dtype: int64

Pierwszym krokiem jest przygotowanie tekstów piosenek do analizy. W tym celu zostaną usunięte znaki interpunkcyjne, a także słowa nie niosące żadnej informacji (stopwords). Następnie tekst zostanie podzielony na pojedyncze słowa (tokenizacja) i zostanie przeprowadzona lematyzacja. Lematyzacja polega na sprowadzeniu słów do ich podstawowej formy.

In [158]:
def clean_text(x):
    x = x.lower()
    x = x.replace('\n', ' ')
    x = x.replace('  ', ' ')
    x = x.replace('-', ' ')
    # delete text in [] brackets
    x = re.sub(r'\[.*?\]', '', x)
    # leave only letters and spaces
    x = re.sub(r'[^a-z\s\'\-]', '', x)
    return x


df['cleaned_text'] = df['lyrics'].apply(lambda x: clean_text(x))


In [159]:
pd.set_option('display.max_colwidth', 1000)
df[['cleaned_text']].head(5)

,cleaned_text
0,he was born in the summer of his th year comin' home to a place hed never been before he left yesterday behind him you might say he was born again you might say he found a key for every door when he first came to the mountains his life was far away on the road and hangin by a song but the strings already broken and he doesnt really care it keeps changin' fast and it dont last for long but the colorado rocky mountain high ive seen it rainin' fire in the sky the shadow from the starlight is softer than a lullaby rocky mountain high colorado rocky mountain high high in colorado he climbed cathedral mountains he saw silver clouds below he saw everything as far as you can see and they say that he got crazy once and he tried to touch the sun and he lost a friend but kept the memory now he walks in quiet solitude the forests and the streams seeking grace in every step he takes his sight has turned inside himself to try and understand the serenity of a clear blue mountain lake and th...
1,there's talk on the street it sounds so familiar great expectations everybody's watching you people you meet they all seem to know you even your old friends treat you like you're something new johnny come lately the new kid in town everybody loves you so don't let them down you look in her eyes the music begins to play hopeless romantics here we go again but after a while you're lookin' the other way it's those restless hearts that never mend oh johnny come lately the new kid in town will she still love you when you're not around there's so many things you should have told her but night after night you're willing to hold her just hold her tears on your shoulder there's talk on the street it's there to remind you doesn't really matter which side you're on you're walking away and they're talking behind you they will never forget you 'til somebody new comes along where you've been lately there's a new kid in town everybody loves him don't they he's holding her and you're still ...
2,hi there how are you it's been a long time seems like we've come a long way my but we learn so slow and heroes they come and they go and leave us behind as if we're supposed to know why why why do we give up our hearts to the past and why must we grow up so fast and all you wishing well fools with your fortunes someone should send you a rose with love from a friend it's nice to hear from you again and the storybook comes to a close gone are the ribbons and bows things to remember places to go pretty maids all in a row
3,when you're out there on your own where your memories can find you like a circle goes around you were lost until you found out what it all comes down to one by one the lonely feelings come day by day they slowly fade away ooh the look was in her eyes you never know what might be found there she was dancing right in time and the moves she made so fine like the music that surrounds her should i stay or go i really want to know would i lose or win if i try and love again oh oh oh oh oh oh oh gonna try and love again oh oh oh oh oh oh oh i'm gonna try and love again oh oh oh oh oh oh oh gonna try and love right or wrong what's done is done it's only moments that you borrow but the thoughts will linger on of the lady and her song when the sun comes up tomorrow well it might take years to see through all these tears don't let go when you find it you will know oh oh oh oh oh oh oh gonna try and love again oh oh oh oh oh oh oh gonna try and love again oh oh oh oh oh oh oh gonna try and lov...
4,nibblin' on sponge cake watchin' the sun bake all of those tourists covered with oil strummin' my six string on my front porch swing smell those shrimp they're beginnin' to boil wastin away again in margaritaville searchin' for my lost shaker of salt some people claim that there's a woman to blame but i know it's nobody's fault don't know the reason i stayed here all season nothin' to show but this brand new tattoo but it's a real beauty a mexican cutie how it 

In [160]:
custom_stopwords = [
    "haha", "ha", "hehe","he", "lol", "yeah", "yo", "hey", "ay","la", "na", "da", "je", "le", "te","oh", "ah", "uh", "eh", "hm", "mm","doo", "dee", "bop", "beep", "sha", "oh", "uh-huh", "huh-huh"
    "woo", "whoa", "oh-oh", "uh-oh", "la-la", "la-la-la","yeah-yeah", "no-no", "ha-ha", "oh-no", "na-na","yeah-yeah-yeah", "la-la-la-la", "na-na-na-na","yeah-yeah-yeah-yeah", "uh-huh", "uh-uh", "hmm", "mmm",
    "yeah-uh", "ooh", "aah", "wow", "yay", "yikes","whoop", "woah", "ugh", "yow", "yep", "yup","oh-yeah", "oh-boy", "ahoy", "bravo", "oy", "oy-vey",
    "whoops", "yoo-hoo", "yahoo", "woop-woop", "gosh", "geez","zap", "zowie", "zing", "bada-bing", "bam","bling", "clink", "kapow", "kerpow", "pow", "splat",
    "swish", "thud", "wham", "zzz","giggle", "snicker", "snort", "chuckle", "snigger", "teehee", "th","there's",
    "hee-haw", "heh-heh", "hohoho", "harr-harr", "haw-haw", "guffaw","blah", "blah-blah", "meh", "bleh", "huh", "duh","fuck", "nigga","nig","jigga", "noah","niggas", "bitch","bitches","hah", "y'all", "pop"
]

In [161]:
#add stopwords
nltk.download('stopwords')
lemmatizer = WordNetLemmatizer()

stop = stopwords.words('english')
stop = stop+custom_stopwords

def tokenize(x):
    #delete stopwords
    x = ' '.join([word for word in x.split() if word not in (stop)])
    return x
 
def lemmatize(x):
    x = ' '.join([lemmatizer.lemmatize(word) for word in x.split()])
    return x


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\zofia\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [162]:
df['tokenize'] = df['cleaned_text'].apply(lambda x: tokenize(x))

In [163]:
df['lemmatize'] = df['tokenize'].apply(lambda x: lemmatize(x))

In [164]:
#pd.set_option('display.max_colwidth', None)
df[ ['lyrics','cleaned_text','tokenize','lemmatize']].head(2)

,lyrics,cleaned_text,tokenize,lemmatize
0,"[Verse 1]\nHe was born in the summer of his 27th year\nComin' home to a place he’d never been before\nHe left yesterday behind him, you might say he was born again\nYou might say he found a key for every door\n\n[Verse 2]\nWhen he first came to the mountains his life was far away\nOn the road and hangin’ by a song\nBut the string’s already broken and he doesn’t really care\nIt keeps changin' fast and it don’t last for long\n\n[Chorus]\nBut the Colorado Rocky Mountain high\nI’ve seen it rainin' fire in the sky\nThe shadow from the starlight is softer than a lullaby\nRocky Mountain high (Colorado)\nRocky Mountain high (High in Colorado)\n\n[Verse 3]\nHe climbed cathedral mountains, he saw silver clouds below\nHe saw everything as far as you can see\nAnd they say that he got crazy once, and he tried to touch the sun\nAnd he lost a friend but kept the memory\n[Verse 4]\nNow he walks in quiet solitude the forests and the streams\nSeeking grace in every step he takes\nHis sight has turne...",he was born in the summer of his th year comin' home to a place hed never been before he left yesterday behind him you might say he was born again you might say he found a key for every door when he first came to the mountains his life was far away on the road and hangin by a song but the strings already broken and he doesnt really care it keeps changin' fast and it dont last for long but the colorado rocky mountain high ive seen it rainin' fire in the sky the shadow from the starlight is softer than a lullaby rocky mountain high colorado rocky mountain high high in colorado he climbed cathedral mountains he saw silver clouds below he saw everything as far as you can see and they say that he got crazy once and he tried to touch the sun and he lost a friend but kept the memory now he walks in quiet solitude the forests and the streams seeking grace in every step he takes his sight has turned inside himself to try and understand the serenity of a clear blue mountain lake and th...,born summer year comin' home place hed never left yesterday behind might say born might say found key every door first came mountains life far away road hangin song strings already broken doesnt really care keeps changin' fast dont last long colorado rocky mountain high ive seen rainin' fire sky shadow starlight softer lullaby rocky mountain high colorado rocky mountain high high colorado climbed cathedral mountains saw silver clouds saw everything far see say got crazy tried touch sun lost friend kept memory walks quiet solitude forests streams seeking grace every step takes sight turned inside try understand serenity clear blue mountain lake colorado rocky mountain high ive seen rainin' fire sky talk god listen casual reply rocky mountain high high colorado rocky mountain high high colorado life full wonder heart still knows fear simple thing cannot comprehend try tear mountains bring couple people scars upon land colorado rocky mountain high ive seen rainin' fire sky though he'd...,born summer year comin' home place hed never left yesterday behind might say born might say found key every door first came mountain life far away road hangin song string already broken doesnt really care keep changin' fast dont last long colorado rocky mountain high ive seen rainin' fire sky shadow starlight softer lullaby rocky mountain high colorado rocky mountain high high colorado climbed cathedral mountain saw silver cloud saw everything far see say got crazy tried touch sun lost friend kept memory walk quiet solitude forest stream seeking grace every step take sight turned inside try understand serenity clear blue mountain lake colorado rocky mountain high ive seen rainin' fire sky talk god listen casual reply rocky mountain high high colorado rocky mountain high high colorado life full wonder heart still know fear simple thing cannot comprehend try tear mountain bring couple people scar upon land colorado rocky mountain high ive seen rainin

In [165]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3600 entries, 0 to 3599
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         3600 non-null   object
 1   tag           3600 non-null   object
 2   artist        3600 non-null   object
 3   year          3600 non-null   int64 
 4   views         3600 non-null   int64 
 5   features      3600 non-null   object
 6   lyrics        3600 non-null   object
 7   id            3600 non-null   int64 
 8   decade        3600 non-null   object
 9   cleaned_text  3600 non-null   object
 10  tokenize      3600 non-null   object
 11  lemmatize     3600 non-null   object
dtypes: int64(3), object(9)
memory usage: 337.6+ KB


In [ ]:
df['lyrcis_preprocessed'] = df['lemmatize']

Aby pokazać różnicę między tekstem przed a po przeprowadzeniu powyższych operacji, policzona została różnica w ilości słów w tekście przed i po przeprowadzeniu powyższych operacji.
Jak widać, po przeprowadzeniu powyższych operacji ilość słów w tekście zmniejszyła się o około 30% w kazdym z tekstów.

In [167]:
df['text_length'] = df['lyrics'].apply(lambda x: len(x))
df['lyrcis_preprocessed_length'] = df['lyrcis_preprocessed'].apply(lambda x: len(x))
df['length_difference'] = df['text_length'] - df['lyrcis_preprocessed_length']

df[['text_length', 'lyrcis_preprocessed_length', 'length_difference']].head()

,text_length,lyrcis_preprocessed_length,length_difference
0,2002,1334,668
1,1653,871,782
2,552,276,276
3,1502,795,707
4,1156,691,465


In [ ]:
df = df.drop(columns=['lemmatize','tokenize','text_length','lyrcis_preprocessed_length','length_difference','cleaned_text'], axis=1)

In [170]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3600 entries, 0 to 3599
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   title                3600 non-null   object
 1   tag                  3600 non-null   object
 2   artist               3600 non-null   object
 3   year                 3600 non-null   int64 
 4   views                3600 non-null   int64 
 5   features             3600 non-null   object
 6   lyrics               3600 non-null   object
 7   id                   3600 non-null   int64 
 8   decade               3600 non-null   object
 9   lyrcis_preprocessed  3600 non-null   object
dtypes: int64(3), object(7)
memory usage: 281.4+ KB


Próbka testowa zostanie zbadana pod kątem sentymentu występującego w piosenkach. W tym celu zostanie wykorzystana biblioteka NLTK. Dla każdej piosenki zostanie obliczony sentyment, a następnie zostanie on przypisany do jednej z kategorii: pozytywny, negatywny, neutralny.

In [171]:
sid = SentimentIntensityAnalyzer()

df['sentiment'] = df['lyrcis_preprocessed'].apply(lambda x: sid.polarity_scores(x))
df['compound'] = df['sentiment'].apply(lambda score_dict: score_dict['compound'])



In [172]:
#get the head of id, title, sentiment and compound columns
df[['title', 'sentiment', 'compound']].head()

,title,sentiment,compound
0,Rocky Mountain High,"{'neg': 0.11, 'neu': 0.824, 'pos': 0.066, 'compound': -0.8498}",-0.8498
1,New Kid in Town,"{'neg': 0.029, 'neu': 0.756, 'pos': 0.215, 'compound': 0.9838}",0.9838
2,Pretty Maids All in a Row,"{'neg': 0.062, 'neu': 0.586, 'pos': 0.353, 'compound': 0.9607}",0.9607
3,Try and Love Again,"{'neg': 0.068, 'neu': 0.663, 'pos': 0.268, 'compound': 0.9905}",0.9905
4,Margaritaville,"{'neg': 0.296, 'neu': 0.609, 'pos': 0.096, 'compound': -0.9758}",-0.9758


In [173]:
def choose_sentiment(compound):
    if compound > 0.3:
        return "Positive"
    elif compound < -0.3:
        return "Negative"
    else:
        return "Neutral"

In [174]:
df['sentiment_category'] = df['compound'].apply(lambda x: choose_sentiment(x) )

Na wykresie poniżej przedstawiono rozkład sentymentu dla próbki testowej.

In [175]:
counts = df.groupby('sentiment_category').size()
print(counts)

sentiment_category
Negative    1056
Neutral      197
Positive    2347
dtype: int64


Kolejnym krokiem była analiza tekstów pod kątem wsytępujacych w nim emocji. W tym celu wykorzystano bibliotekę text2emotion. Dla każdej piosenki zostały obliczone emocje, a następnie zostały one przypisane do jednej z kategorii: radość, smutek, złość, strach.

In [176]:
df['emotion'] = df['lyrcis_preprocessed'].apply(lambda x: get_emotion(x))

df[['title', 'emotion']].head()

,title,emotion
0,Rocky Mountain High,"{'Happy': 0.04, 'Angry': 0.01, 'Surprise': 0.05, 'Sad': 0.45, 'Fear': 0.45}"
1,New Kid in Town,"{'Happy': 0.39, 'Angry': 0.03, 'Surprise': 0.1, 'Sad': 0.42, 'Fear': 0.05}"
2,Pretty Maids All in a Row,"{'Happy': 0.43, 'Angry': 0.0, 'Surprise': 0.0, 'Sad': 0.14, 'Fear': 0.43}"
3,Try and Love Again,"{'Happy': 0.52, 'Angry': 0.1, 'Surprise': 0.0, 'Sad': 0.29, 'Fear': 0.1}"
4,Margaritaville,"{'Happy': 0.04, 'Angry': 0.23, 'Surprise': 0.31, 'Sad': 0.31, 'Fear': 0.12}"


In [177]:
pd.set_option('display.max_colwidth', 100)
df.head(2)

,title,tag,artist,year,views,features,lyrics,id,decade,lyrcis_preprocessed,sentiment,compound,sentiment_category,emotion
0,Rocky Mountain High,country,John Denver,1972,34529,{},[Verse 1]\nHe was born in the summer of his 27th year\nComin' home to a place he’d never been be...,2873,1970s,born summer year comin' home place hed never left yesterday behind might say born might say foun...,"{'neg': 0.11, 'neu': 0.824, 'pos': 0.066, 'compound': -0.8498}",-0.8498,Negative,"{'Happy': 0.04, 'Angry': 0.01, 'Surprise': 0.05, 'Sad': 0.45, 'Fear': 0.45}"
1,New Kid in Town,country,Eagles,1976,76529,{},"[Verse 1]\nThere's talk on the street, it sounds so familiar\nGreat expectations, everybody's wa...",54282,1970s,talk street sound familiar great expectation everybody's watching people meet seem know even old...,"{'neg': 0.029, 'neu': 0.756, 'pos': 0.215, 'compound': 0.9838}",0.9838,Positive,"{'Happy': 0.39, 'Angry': 0.03, 'Surprise': 0.1, 'Sad': 0.42, 'Fear': 0.05}"


In [178]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3600 entries, 0 to 3599
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   title                3600 non-null   object 
 1   tag                  3600 non-null   object 
 2   artist               3600 non-null   object 
 3   year                 3600 non-null   int64  
 4   views                3600 non-null   int64  
 5   features             3600 non-null   object 
 6   lyrics               3600 non-null   object 
 7   id                   3600 non-null   int64  
 8   decade               3600 non-null   object 
 9   lyrcis_preprocessed  3600 non-null   object 
 10  sentiment            3600 non-null   object 
 11  compound             3600 non-null   float64
 12  sentiment_category   3600 non-null   object 
 13  emotion              3600 non-null   object 
dtypes: float64(1), int64(3), object(10)
memory usage: 393.9+ KB


In [179]:
df.to_csv('data_sentiment_emotion_3600.csv', index=False)